## Steps for generating YoloNAS dlc
#### Note->Use python3.8 or above for generating onnx and python3.6 for generating dlc

In [1]:
## Note- Use python3.8 or above for generating onnx

!pip install super-gradients==3.1.2


## Downloading Model from git repo
import torch
# Load model with pretrained weights
from super_gradients.training import models
from super_gradients.common.object_names import Models

model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")

# Prepare model for conversion
# Input size is in format of [Batch x Channels x Width x Height] where 640 is the standard COCO dataset dimensions
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, 320, 320])

# Create dummy_input
dummy_input = torch.randn([1, 3, 320, 320], device="cpu")

# Convert model to onnx
torch.onnx.export(model, dummy_input, "yolo_nas_s.onnx", opset_version=11)

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-07-13 17:15:54] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2023-07-13 17:15:57] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-07-13 17:15:57] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-07-13 17:15:57] WARNING - export.py - Failed to import pytorch_quantization
[2023-07-13 17:15:57] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-07-13 17:15:58] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


#### Enable python3.6 environment, to use SNPE SDK and then convert onnx to dlc

In [2]:
%%bash
snpe-onnx-to-dlc -i yolo_nas_s.onnx -o app/src/main/assets/yolo_nas_s.dlc

2023-07-13 17:16:03,073 - 235 - INFO - Successfully simplified the onnx model in child process
2023-07-13 17:16:03,547 - 235 - INFO - Successfully receive the simplified onnx model in main process
2023-07-13 17:16:06,272 - 235 - INFO - INFO_INITIALIZATION_SUCCESS: 


## Quantizing MobileNetSSD

In [27]:
##STEPS to preprocess images

def preprocess(original_image):
    resized_image = cv2.resize(original_image, (320, 320))
    resized_image = resized_image/255
    return resized_image

import cv2
import numpy as np
import os

##Please download Coco2014 dataset and give the path here
dataset_path = "/workspace/val2014/"

!mkdir -p rawYoloNAS

filenames=[]
for path in os.listdir(dataset_path)[:5]:
    # check if current path is a file
    if os.path.isfile(os.path.join(dataset_path, path)):
        filenames.append(os.path.join(dataset_path, path))

for filename in filenames:
    original_image = cv2.imread(filename)
    img = preprocess(original_image)
    img = img.astype(np.float32)
    img.tofile("rawYoloNAS/"+filename.split("/")[-1].split(".")[0]+".raw")

In [19]:
%%bash
find rawYoloNAS -name *.raw > YoloInputlist.txt

In [20]:
%%bash
cat YoloInputlist.txt

rawYoloNAS/COCO_val2014_000000000400.raw
rawYoloNAS/COCO_val2014_000000000042.raw
rawYoloNAS/COCO_val2014_000000000073.raw
rawYoloNAS/COCO_val2014_000000000074.raw
rawYoloNAS/COCO_val2014_000000000133.raw


In [21]:
%%bash
snpe-dlc-quantize --input_dlc app/src/main/assets/yolo_nas_s.dlc --input_list YoloInputlist.txt --use_enhanced_quantizer --use_adjusted_weights_quantizer --axis_quant --output_dlc app/src/main/assets/Quant_yoloNas_s_320.dlc

[INFO] InitializeStderr: DebugLog initialized.
[INFO] Processed command-line arguments
[INFO] Quantized parameters
[INFO] Generated activations
[INFO] Saved quantized dlc to: app/src/main/assets/Quant_yoloNas_s_320.dlc
[INFO] DebugLog shutting down.


     0.1ms [  INFO ] Initializing logging in the backend. Callback: [0xc42410], Log Level: [3]
     0.1ms [  INFO ] No BackendExtensions lib provided;initializing NetRunBackend Interface
   875.4ms [  INFO ] cleaning up resources for input tensors
   875.5ms [  INFO ] cleaning up resources for output tensors
  1524.6ms [  INFO ] cleaning up resources for input tensors
  1524.6ms [  INFO ] cleaning up resources for output tensors
  2136.3ms [  INFO ] cleaning up resources for input tensors
  2136.3ms [  INFO ] cleaning up resources for output tensors
  2852.1ms [  INFO ] cleaning up resources for input tensors
  2852.1ms [  INFO ] cleaning up resources for output tensors
  3461.0ms [  INFO ] cleaning up resources for input tensors
  3461.0ms [  INFO ] cleaning up resources for output tensors
